<a href="https://colab.research.google.com/github/Darshan59063/Chat-Bot/blob/main/Copy_of_Xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

In [ ]:
base_dir = '/content/drive/MyDrive/Chest_xray '
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')


# Directory with our training cat pictures
train_nor_dir = os.path.join(train_dir, 'NORMAL')

# Directory with our training dog pictures
train_pne_dir = os.path.join(train_dir, 'PNEUMONIA')

# Directory with our Testing cat pictures
Testing_nor_dir = os.path.join(validation_dir, 'NORMAL')

# Directory with our Testing dog pictures
Testing_pne_dir = os.path.join(validation_dir, 'PNEUMONIA')


Test_nor_dir = os.path.join(test_dir, 'NORMAL')

# Directory with our Testing dog pictures
Test_pne_dir = os.path.join(test_dir, 'PNEUMONIA')


In [ ]:
print('total training cat images:', len(os.listdir(train_nor_dir)))
print('total training dog images:', len(os.listdir(train_pne_dir )))
print('total Testing cat images:', len(os.listdir(Testing_nor_dir)))
print('total Testing dog images:', len(os.listdir(Testing_pne_dir)))
print('total Testing cat images:', len(os.listdir(Test_nor_dir)))
print('total Testing dog images:', len(os.listdir(Test_pne_dir)))

X_train = len(os.listdir(train_dir))
X_val = len(os.listdir(validation_dir))

total training cat images: 1341
total training dog images: 3875
total Testing cat images: 8
total Testing dog images: 8
total Testing cat images: 234
total Testing dog images: 390


In [ ]:
from keras import layers
from keras import Model

img_input = layers.Input(shape=(150, 150, 3))
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)


x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)


x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)


x = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(x)

output = layers.Dense(1, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)




from keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')


# Flow validation images in batches of 20 using val_datagen generator
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),  # All images will be resized to 150x150
        class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch = 163,  # 5216 images = 32 * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps = 1, # 16 images = 20 * steps
      verbose=2)


Epoch 1/15
163/163 - 1400s - loss: 0.2173 - acc: 0.9197 - val_loss: 0.2068 - val_acc: 0.9375 - 1400s/epoch - 9s/step
Epoch 2/15
163/163 - 170s - loss: 0.1370 - acc: 0.9515 - val_loss: 0.1886 - val_acc: 0.8750 - 170s/epoch - 1s/step
Epoch 3/15
163/163 - 168s - loss: 0.1101 - acc: 0.9661 - val_loss: 0.5934 - val_acc: 0.8125 - 168s/epoch - 1s/step
Epoch 4/15
163/163 - 173s - loss: 0.0823 - acc: 0.9730 - val_loss: 0.2010 - val_acc: 0.8750 - 173s/epoch - 1s/step
Epoch 5/15
163/163 - 169s - loss: 0.0611 - acc: 0.9789 - val_loss: 0.5808 - val_acc: 0.8125 - 169s/epoch - 1s/step
Epoch 6/15
163/163 - 168s - loss: 0.0464 - acc: 0.9831 - val_loss: 0.0802 - val_acc: 0.9375 - 168s/epoch - 1s/step
Epoch 7/15
163/163 - 173s - loss: 0.0414 - acc: 0.9858 - val_loss: 0.1219 - val_acc: 0.9375 - 173s/epoch - 1s/step
Epoch 8/15
163/163 - 168s - loss: 0.0383 - acc: 0.9887 - val_loss: 0.4084 - val_acc: 0.8125 - 168s/epoch - 1s/step
Epoch 9/15
163/163 - 170s - loss: 0.0226 - acc: 0.9906 - val_loss: 0.0459 - va

In [ ]:
model.evaluate( test_generator, batch_size=20, verbose=2 )